代码作用：把输入的 token 序列做词向量嵌入（token embedding）并加上位置嵌入（positional embedding）。

In [1]:
import torch

class Embed(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.embed = torch.nn.Embedding(49408, 768)
        # 定义词嵌入层（可训练参数）。
        # 形状说明：Embedding(num_embeddings, embedding_dim)
        # num_embeddings=49408：词表大小（能索引的 token 个数）。
        # embedding_dim=768：每个 token 的向量维度。

        self.pos_embed = torch.nn.Embedding(77, 768)
        # 定义位置嵌入层（同样可训练）。
        # 这里序列最大长度设为 77，所以位置索引范围是 [0, 76]，每个位置一条 768 维向量。

        self.register_buffer('pos_ids', torch.arange(77).unsqueeze(dim=0))
        # 注册一个名为 pos_ids 的buffer（不是参数，没有梯度，但会随模型保存/加载、随 .to(device) 自动搬运到同一设备）。
        # torch.arange(77) 生成 [0, 1, 2, ..., 76]，形状 [77]。
        # .unsqueeze(dim=0) 变成形状 [1, 77]，方便后续与 batch 做广播相加。

    def forward(self, input_ids): # 前向函数，input_ids 是输入的 token 索引张量。
        #input_ids -> [b, 77] 约定输入的序列长度是 77（与位置嵌入一致）。batch 大小记为 b。

        embed = self.embed(input_ids) # 查词嵌入。 形状变化：[b, 77] → [b, 77, 768]。

        pos_embed = self.pos_embed(self.pos_ids) # 用事先存好的 pos_ids（形状 [1, 77]）查位置嵌入。 形状变化：[1, 77] → [1, 77, 768]。 注意这里之所以把 pos_ids 做成 [1, 77]，就是为了后面能在 batch 维上广播。

        return embed + pos_embed # 把词嵌入与位置嵌入相加得到最终嵌入。 由于 embed 是 [b, 77, 768]，pos_emb 是 [1, 77, 768]，广播后相加结果是 [b, 77, 768]。


Embed()(torch.ones(2, 77).long()).shape # 建一个 Embed 实例，并用一个形状为 [2, 77] 的整型输入测试（这里用的是全 1 的 fake 索引）。 输出形状应为 [2, 77, 768]，与上面的推导一致。

# “有 2 句话 × 每句话 77 个位置 × 每个位置一个 768 维向量”。

torch.Size([2, 77, 768])

encoder注意力层 典型的多头注意力 计算过程mask(q*k)*v 带向后的注意力mask

In [2]:
class Atten(torch.nn.Module):
    """
    定义了一个最小可用的多头自注意力（Multi-Head Self-Attention）模块的核心。
    共有四个线性层：
    q/k/v：把输入特征投影到查询/键/值空间（维度都还是 768）。
    out：把多头拼接后的结果再映射回 768 维。
    这里没有显式的 nn.MultiheadAttention，而是手动实现多头拆分与计算。
    """
    def __init__(self):
        super().__init__() 
        self.q = torch.nn.Linear(768, 768)
        self.k = torch.nn.Linear(768, 768)
        self.v = torch.nn.Linear(768, 768)
        self.out = torch.nn.Linear(768, 768)

    def forward(self, x):
        #x is input sequence length -> [b, 77, 768]

        b = x.shape[0] # batch size

        """
        通过三组线性层得到 q/k/v，形状仍是 [b, 77, 768]。
        关键点：q 乘了 0.125。
        这是缩放因子，等于 1 / sqrt(d_head)。本实现中单头维度稍后会是 64，所以 1/sqrt(64)=1/8=0.125。
        作用：与标准 Scaled Dot-Product Attention 一致，避免点积值过大导致 softmax 退化/梯度不稳定。
        """
        #维度不变,得到q,k,v三个矩阵
        #[b, 77, 768]
        q = self.q(x) * 0.125
        k = self.k(x)
        v = self.v(x)

        """
        把 768 维拆成 12 个头、每头 64 维（因为 12*64=768）。
        步骤说明：
        reshape(b, 77, 12, 64)：显式引入头数维度。
        transpose(1, 2)：把维度变为 [b, 12, 77, 64]，即先按头再按序列。
        reshape(b*12, 77, 64)：把 batch 和 head 合并，方便用 torch.bmm 做批量矩阵乘法。
        这样每个头就像一个“独立样本”。
        """
        #拆分注意力头
        #[b, 77, 768] -> [b, 77, 12, 64] -> [b, 12, 77, 64] -> [b*12, 77, 64]
        q = q.reshape(b, 77, 12, 64).transpose(1, 2).reshape(b * 12, 77, 64)
        k = k.reshape(b, 77, 12, 64).transpose(1, 2).reshape(b * 12, 77, 64)
        v = v.reshape(b, 77, 12, 64).transpose(1, 2).reshape(b * 12, 77, 64)

        #计算qk乘积 对每个头做 Q @ K^T，得到注意力分数矩阵（未经 softmax）。 每个头得到一个 [77, 77] 的相关性矩阵：第 i 个 token 对第 j 个 token 的注意力打分。
        #[b*12, 77, 64] * [b*12, 64, 77] -> [b*12, 77, 77] 
        attn = torch.bmm(q, k.transpose(1, 2))

        #[b*12, 77, 77] -> [b, 12, 77, 77] 把合并的 b*12 维拆回 [b, 12, ...]，以便加 mask。
        attn = attn.reshape(b, 12, 77, 77)

        """
        生成一个形状为 [b, 1, 77, 77] 的 mask，用于因果（下三角）注意力：
        fill_(-inf) 先全填为 -∞。
        triu_(1)（保留主对角线上方的上三角，其他置 0）：
        结果是严格上三角部分为 -∞，主对角线及其下方为 0。
        含义：位置 t 只能看见自己以及之前的位置（对角线及以下），不允许看“未来”（上三角被设为 -∞）。
        """
        #覆盖mask
        def get_mask(b):
            mask = torch.empty(b, 77, 77)

            #上三角的部分置为负无穷
            mask.fill_(-float('inf'))

            #对角线和以下的位置为0
            mask.triu_(1)

            return mask.unsqueeze(1) # [b, 77, 77] - > [b, 1, 77, 77]

        """
        把 mask 加到注意力分数上：
        上三角加上 -∞，在后续 softmax 里会变成 0 概率（被完全屏蔽）。
        对角线及以下位置加 0，不受影响。
        .to(attn.device) 确保与 attn 在同一设备（CPU/GPU）上。
        """
        #[b, 12, 77, 77] + [b, 1, 77, 77] -> [b, 12, 77, 77]
        attn = attn + get_mask(attn.shape[0]).to(attn.device)

        #[b, 12, 77, 77] -> [b*12, 77, 77] 再次把 batch 和 head 合并，方便用 bmm。
        attn = attn.reshape(b * 12, 77, 77)

        #计算softmax,被mask的部分值为0
        attn = attn.softmax(dim=-1) # 在最后一个维度（键的维度）做 softmax，得到每个查询位置对所有键位置的注意力权重。
        # 由于上三角是 -∞，softmax 后这些位置权重为 0，实现因果屏蔽。

        #计算和v的乘积
        #[b*12, 77, 77] * [b*12, 77, 64] -> [b*12, 77, 64] 用注意力权重对 V 做加权求和，得到每个头的上下文向量。
        attn = torch.bmm(attn, v)

        """
        把头维合回来：
        先还原为 [b, 12, 77, 64]
        转置为 [b, 77, 12, 64]（更便于拼接）
        最后拼成 [b, 77, 768]（12 头 × 64 维 = 768）
        """
        #[b*12, 77, 64] -> [b, 12, 77, 64] -> [b, 77, 12, 64] -> [b, 77, 768]
        attn = attn.reshape(b, 12, 77, 64).transpose(1, 2).reshape(b, 77, 768)

        #线性输出,维度不变
        #[b, 77, 768]
        return self.out(attn)


Atten()(torch.randn(2, 77, 768)).shape

torch.Size([2, 77, 768])

一层编码器
激活函数quick gelu:
x * sigmoid(1.702 * x)

Transformer 的一个“预归一化（Pre-LN）Block”：自注意力残差块 + 前馈（MLP）残差块，激活使用 QuickGELU。

In [3]:
class ClipEncoder(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.s1 = torch.nn.Sequential(
            torch.nn.LayerNorm(768),
            Atten(),
        )

        self.s2 = torch.nn.Sequential(
            torch.nn.LayerNorm(768),
            torch.nn.Linear(768, 3072),
        )

        self.s3 = torch.nn.Linear(3072, 768)

    def forward(self, x):
        #x -> [2, 77, 768]

        #维度不变
        #[2, 77, 768]
        x = x + self.s1(x)

        #[2, 77, 768]
        res = x

        #[2, 77, 768] -> [2, 77, 3072]
        x = self.s2(x)

        #维度不变
        #[2, 77, 3072]
        x = x * (x * 1.702).sigmoid()

        #[2, 77, 3072] -> [2, 77, 768]
        return res + self.s3(x)


ClipEncoder()(torch.randn(2, 77, 768)).shape

torch.Size([2, 77, 768])

In [4]:
#经过优化之后的代码量少得吓人...
encoder = torch.nn.Sequential(
    Embed(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    ClipEncoder(),
    torch.nn.LayerNorm(768),
)

encoder(torch.ones(2, 77).long()).shape

torch.Size([2, 77, 768])

加载预训练参数

现在我们有了encoder模型，但是它当中所有的参数还是随机初始化的，为了帮助我们的训练过程能够更快的进行收敛，我们需要从一个已经训练的模型当中，把这些参数给加载过来，这里我们从这个checkpoint来加载一个预训练的模型，然后从这个预训练的模型当中把它所有的参数给它抽取出来，抽取到我们自己的模型当中，我们用预训练模型当中的参数来初始化我们自己的模型。

In [ ]:
from transformers import CLIPTextModel # 从 transformers（Hugging Face）库中导入 CLIPTextModel，它是 CLIP 文本编码器（Transformer 堆叠）的实现与权重加载入口。

#加载预训练模型的参数 
"""
从 Hugging Face Hub 仓库 'lansinuote/diffsion_from_scratch.params' 的子文件夹 text_encoder 下载并实例化一个 CLIPTextModel。
变量名叫 params，但它其实是完整模型实例（包含子模块与参数），不是“纯粹的参数字典”。
典型 CLIP 文本分支结构：Token Embedding → Position Embedding → 12 层 Transformer Encoder Block → Final LayerNorm。你的后续代码就是把这些预训练权重逐模块拷贝到你自定义的 encoder 结构中。
"""
params = CLIPTextModel.from_pretrained(
    'lansinuote/diffsion_from_scratch.params', subfolder='text_encoder')

"""
将 Hugging Face 模型中 token embedding（形如 [vocab_size, hidden_size]，常见 [vocab, 768]）的权重，拷贝到你自定义的 encoder[0].embed。
load_state_dict 会严格匹配参数名与形状；这里你是对子模块调用，等价于把该子模块的权重一一覆盖。
"""
#词编码
encoder[0].embed.load_state_dict(
    params.text_model.embeddings.token_embedding.state_dict())
#位置编码
encoder[0].pos_embed.load_state_dict(
    params.text_model.embeddings.position_embedding.state_dict())

#12层编码层
"""
循环 12 次（i = 0..11），对应 CLIP 文本编码器的 12 层 Transformer Block。
你后续访问的是 encoder[i + 1]，因此层索引映射为：
预训练第 0 层 → 你的 encoder[1]
预训练第 11 层 → 你的 encoder[12]
"""
for i in range(12):

    #第一层norm 
    encoder[i + 1].s1[0].load_state_dict(
        params.text_model.encoder.layers[i].layer_norm1.state_dict())

    #注意力q矩阵
    encoder[i + 1].s1[1].q.load_state_dict(
        params.text_model.encoder.layers[i].self_attn.q_proj.state_dict())

    #注意力k矩阵
    encoder[i + 1].s1[1].k.load_state_dict(
        params.text_model.encoder.layers[i].self_attn.k_proj.state_dict())

    #注意力v矩阵
    encoder[i + 1].s1[1].v.load_state_dict(
        params.text_model.encoder.layers[i].self_attn.v_proj.state_dict())

    #注意力out
    encoder[i + 1].s1[1].out.load_state_dict(
        params.text_model.encoder.layers[i].self_attn.out_proj.state_dict())

    #第二层norm
    encoder[i + 1].s2[0].load_state_dict(
        params.text_model.encoder.layers[i].layer_norm2.state_dict())

    #mlp第一层fc
    encoder[i + 1].s2[1].load_state_dict(
        params.text_model.encoder.layers[i].mlp.fc1.state_dict())

    #mlp第二层fc
    encoder[i + 1].s3.load_state_dict(
        params.text_model.encoder.layers[i].mlp.fc2.state_dict())

#输出norm
encoder[13].load_state_dict(params.text_model.final_layer_norm.state_dict())

d:\Anaconda\envs\torch_env\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

d:\Anaconda\envs\torch_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yingz\.cache\huggingface\hub\models--lansinuote--diffsion_from_scratch.params. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regul

pytorch_model.bin:   0%|          | 0.00/492M [00:00<?, ?B/s]

<All keys matched successfully>

In [6]:
# a = encoder(torch.arange(77).unsqueeze(dim=0))
# b = params(torch.arange(77).unsqueeze(dim=0)).last_hidden_state

# (a == b).all()